In [46]:
import pandas as pd
import json

In [47]:
df = pd.read_csv('dpc-covid19-ita-regioni.csv')
df_pop = pd.read_csv('DCIS_POPORESBIL1_12012022143315331.csv')
df_cod = pd.read_csv('Codici-statistici-e-denominazioni-al-01_01_2022.csv', sep=';',encoding='latin-1')
json_r = 'limits_IT_regions.geojson'

In [48]:
df_pm10 = pd.read_csv('DataExtract.csv')
df_net_r = pd.read_csv('air_quality_net_italy_regions.csv', sep=';')

In [28]:
df_sup = pd.read_csv('DCCV_CARGEOMOR_ST_COM_27032022165808849.csv')

In [64]:
df_sup = df_sup.rename(columns={'Territorio': 'region_name', 'Value': 'surface_km2'})
df_sup = df_sup[['region_name', 'surface_km2']]
df_sup.at[1,'region_name']='Valle d\'Aosta/Vallée d\'Aoste'
df_sup.at[4,'region_name']='Trentino-Alto Adige/Südtirol'

In [65]:
df_sup

,region_name,surface_km2
0,Piemonte,25386.6971
1,Valle d'Aosta/Vallée d'Aoste,3260.8546
2,Liguria,5416.1520
3,Lombardia,23863.0974
4,Trentino-Alto Adige/Südtirol,13604.7213
5,Veneto,18345.3695
6,Friuli-Venezia Giulia,7932.4822
7,Emilia-Romagna,22444.5420
8,Toscana,22987.4371
9,Umbria,8464.2233


In [49]:
# converting values in the "data" column in pandas datetime values
datetimes = pd.to_datetime(df["data"])
df["data"] = datetimes

In [50]:
#filtering rows on "data" column
#only keeping rows with "2020-05-24 17:00:00" value
#to only get death and cases up until the 24th of may of 2020
start_date = "2020-05-24 17:00:00"
end_date = "2020-05-24 17:00:00"
after_start_date = df["data"] >= start_date
before_end_date = df["data"] <= end_date
between_two_dates = after_start_date & before_end_date
filtered_dates = df.loc[between_two_dates]

df = filtered_dates

In [51]:
#removing unnecessary columns
df_tot_pos_morti25052020 = df[['codice_regione', 'denominazione_regione', 'totale_positivi', 'deceduti']]

#ordering rows based on region code
df_tot_pos_morti25052020.sort_values(by=['codice_regione'], inplace=True)
df_tot_pos_morti25052020=df_tot_pos_morti25052020.reset_index(drop=True)

/usr/local/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [52]:
#merging P.A. Bolzano and P.A. Trento
tot_pos_trentino = df_tot_pos_morti25052020['totale_positivi'].values[19] + df_tot_pos_morti25052020['totale_positivi'].values[20]
deceduti_trentino = df_tot_pos_morti25052020['deceduti'].values[19] + df_tot_pos_morti25052020['deceduti'].values[20]

trentino = pd.DataFrame({'codice_regione':4, 'denominazione_regione':'Trentino-Alto Adige/Südtirol', 'totale_positivi':tot_pos_trentino, 'deceduti':deceduti_trentino}, index=[3])
df_tot_pos_morti25052020 = pd.concat([df_tot_pos_morti25052020.iloc[:3], trentino, df_tot_pos_morti25052020.iloc[3:]]).reset_index(drop=True)
df_tot_pos_morti25052020 = df_tot_pos_morti25052020.drop([20, 21])

In [53]:
#cleaning dataset regarding population numbers
df_pop = df_pop.loc[df_pop['TIME'] == '2020']
df_pop.reset_index(drop=True)
df_pop = df_pop[['Territorio', 'Value']]
df_pop = df_pop.rename(columns={'Territorio': 'denominazione_regione', 'Value': 'popolazione'})
df_pop = df_pop.reset_index(drop=True)
df_pop['denominazione_regione'] = df_pop['denominazione_regione'].replace({'Valle d\'Aosta / Vallée d\'Aoste':'Valle d\'Aosta', 'Friuli-Venezia Giulia':'Friuli Venezia Giulia'})

In [54]:
# cleaning pm10 ds
df_pm10_r = pd.merge(df_pm10, df_net_r, on="Air Quality Network")
pm10r_fil = df_pm10_r[['Air Pollution Level', 'Region Code']]
pm10r_fil = pm10r_fil.groupby('Region Code', as_index=False)['Air Pollution Level'].mean().round(2)
pm10r_fil = pm10r_fil.rename(columns={'Region Code': 'region_istat_code', 'Air Pollution Level': 'pm10_mean_level'})
pm10r_fil = pm10r_fil.reset_index(drop=True)

In [55]:
#merging P.A. Bolzano and P.A. Trento
pop_trentino = df_pop['popolazione'].values[4] + df_pop['popolazione'].values[5]
trentino = pd.DataFrame({'denominazione_regione':'Trentino-Alto Adige/Südtirol', 'popolazione':pop_trentino}, index=[3])
df_pop = pd.concat([df_pop.iloc[:3], trentino, df_pop.iloc[3:]]).reset_index(drop=True)
df_pop = df_pop.drop([5, 6])

In [56]:
#merging of population dataset with covid dataset
df_finale = pd.merge(df_tot_pos_morti25052020, df_pop, on="denominazione_regione")

In [57]:
#renaming regions with istat names
df_cod = df_cod[['Denominazione Regione']]
df_cod = df_cod.drop_duplicates()
df_cod = df_cod.reset_index(drop=True)
df_cod

df_finale['denominazione_regione'] = df_cod['Denominazione Regione']

In [58]:
#adding column regarding number of cases and death for every 100000 residents
df_finale['totale_positivi_ogni_100000'] = df_finale.apply(lambda row: round((row.totale_positivi / row.popolazione)*100000), axis=1)
df_finale['deceduti_ogni_100000'] = df_finale.apply(lambda row: round((row.deceduti / row.popolazione)*100000), axis=1)

In [59]:
#translation of columns in english
df_finale = df_finale.rename(columns={'codice_regione': 'region_istat_code', 'denominazione_regione': 'region_name', 'totale_positivi': 'tot_positives', 'deceduti': 'deaths', 'popolazione':'population', 'totale_positivi_ogni_100000':'tot_positives_every_100000', 'deceduti_ogni_100000': 'deaths_every_100000'})

In [60]:
#merging pm10 ds with the final ds 
df_finale = pd.merge(df_finale, pm10r_fil, on="region_istat_code")

In [61]:
df_finale

,region_istat_code,region_name,tot_positives,deaths,population,tot_positives_every_100000,deaths_every_100000,pm10_mean_level
0,1,Piemonte,7703,3783,4311217,179,88,22.84
1,2,Valle d'Aosta/Vallée d'Aoste,32,143,125034,26,114,16.98
2,3,Lombardia,25614,15840,10027602,255,158,27.93
3,4,Trentino-Alto Adige/Südtirol,730,748,1078069,68,69,16.57
4,5,Veneto,2660,1869,4879133,55,38,28.97
5,6,Friuli-Venezia Giulia,412,329,1206216,34,27,19.24
6,7,Liguria,1624,1419,1524826,107,93,19.46
7,8,Emilia-Romagna,4457,4055,4464119,100,91,24.94
8,9,Toscana,1700,1013,3692555,46,27,20.74
9,10,Umbria,53,75,870165,6,9,20.06


In [66]:
df_finale = pd.merge(df_finale, df_sup)
df_finale['population_density'] = df_finale.apply(lambda row: round((row.population / row.surface_km2)), axis=1)

In [67]:
#creating csv file
df_finale_csv = df_finale.to_csv('covid-data-regions-first-quarantine.csv', encoding='utf-8', index=False)

In [68]:
#creating json file
df_dict = df_finale.to_dict(orient='records')

with open('limits_IT_regions.geojson', 'r') as f1:
    json_rs = json.load(f1)

jreg = json_rs.copy()

for feat in jreg['features']:
    f_pr = feat['properties'].copy()
    for i in df_dict:
        if f_pr['reg_istat_code_num'] == i['region_istat_code']:
            feat['properties'].clear()
            feat['properties'].update(i)
            
with open('covid-data-regions-first-quarantine.geojson', 'w') as f:
    json.dump(jreg, f)